In [10]:
import json
import numpy as np
import networkx as nx
import tqdm

# read topo data
with open('data/resultData_cleaned_fixed.json','r') as f:
    data = json.load(f)

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']
NUM_NODES, NUM_EDGES = len(nodes), len(edges)

# creating some usefull maps to get information regarding the nodes
new_to_old = {}
for node in nodes:
    new_to_old[node['id']] = node['old_id']

id_to_node = {}
for node in nodes:
    id_to_node[node['id']] = node

port_map = {}
for node in nodes:
    port_map[node['id']] = {}
    for port in node['portList']:
        port_map[node['id']][port['id']] = port

for edge in edges:
    edge['srcOld'] = new_to_old[edge['srcNode']]
    edge['destOld'] = new_to_old[edge['destNode']]


g = nx.DiGraph()

for edge in edges:
    source, sport, dest, dport = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']

    sp_bw = port_map[source][sport]['bw']
    dp_bw = port_map[dest][dport]['bw']

    g.add_edge(source, dest, bw=min(sp_bw, dp_bw))


In [11]:
# get the nodes of the subgraphs in the topology
subgraphs = list(nx.strongly_connected_components(g))

ids = list(subgraphs[0]) + list(subgraphs[1]) + list(subgraphs[2])

In [20]:
count_loops = 0
for edge in edges:
    if edge['srcNode'] == edge['destNode'] and edge['srcNode'] in ids:
        count_loops += 1
        
count_loops

60

In [8]:
reindex_map = {}

for new_id, old_id in enumerate(ids):
    reindex_map[new_to_old[old_id]] = new_id

nodes_to_keep = list(reindex_map.keys())

new_nodes, new_edges = [], []

for node in nodes:
    if node['old_id'] in nodes_to_keep:
        new_nodes.append(node)


for edge in edges:
    if edge['srcOld'] in nodes_to_keep and edge['destOld'] in nodes_to_keep:
        new_edges.append(edge)


from copy import deepcopy

nodes, edges = [], []

while new_nodes:
    node = deepcopy(new_nodes.pop(0))
    node['id'] = reindex_map[node['old_id']]
    nodes.append(node)

while new_edges:
    edge = deepcopy(new_edges.pop(0))
    edge['destNode'] = reindex_map[edge['destOld']]
    edge['srcNode'] = reindex_map[edge['srcOld']]
    edges.append(edge)

id_to_node = {}
for node in nodes:
    id_to_node[node['id']] = node

port_map = {}
for node in nodes:
    port_map[node['id']] = {}
    for port in node['portList']:
        port_map[node['id']][port['id']] = port


g = nx.DiGraph()

for edge in edges:
    source, sport, dest, dport = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']

    sp_bw = port_map[source][sport]['bw']
    dp_bw = port_map[dest][dport]['bw']

    g.add_edge(source, dest, bw=min(sp_bw, dp_bw))

In [9]:
# get the nodes of the subgraphs in the topology
subgraphs = list(nx.strongly_connected_components(g))

ids = list(subgraphs[0]) + list(subgraphs[1]) + list(subgraphs[2])

reindex_map = {}

for new_id, old_id in enumerate(ids):
    reindex_map[new_to_old[old_id]] = new_id

nodes_to_keep = list(reindex_map.keys())

new_nodes, new_edges = [], []

for node in nodes:
    if node['old_id'] in nodes_to_keep:
        new_nodes.append(node)


for edge in edges:
    if edge['srcOld'] in nodes_to_keep and edge['destOld'] in nodes_to_keep:
        new_edges.append(edge)


from copy import deepcopy

nodes, edges = [], []

while new_nodes:
    node = deepcopy(new_nodes.pop(0))
    node['id'] = reindex_map[node['old_id']]
    nodes.append(node)

while new_edges:
    edge = deepcopy(new_edges.pop(0))
    edge['destNode'] = reindex_map[edge['destOld']]
    edge['srcNode'] = reindex_map[edge['srcOld']]
    edges.append(edge)

id_to_node = {}
for node in nodes:
    id_to_node[node['id']] = node

port_map = {}
for node in nodes:
    port_map[node['id']] = {}
    for port in node['portList']:
        port_map[node['id']][port['id']] = port


g = nx.DiGraph()

for edge in edges:
    source, sport, dest, dport = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']

    sp_bw = port_map[source][sport]['bw']
    dp_bw = port_map[dest][dport]['bw']

    g.add_edge(source, dest, bw=min(sp_bw, dp_bw))

56

In [4]:
with open("data/reindexed.json", 'w+') as f:
    json.dump(
        {'nodeList': nodes, 'edgeList': edges},
        f,
        indent=4
    )